In [2]:
import requests
import shutil
import os
import io
import json
import time
import datetime
import dateutil
import re
from IPython.display import clear_output
from pprint import pprint

parent_folder='steam_data'

if not os.path.exists(parent_folder):
    os.makedirs(parent_folder)

In [72]:
with open(os.path.join(parent_folder,'steamspy_exearlyaccess.html'),'r') as file:
    temphtml=file.read()
all_games=re.findall(r'<td data-order=.+<a href=/app/[0-9]+',temphtml)
with open(os.path.join(parent_folder,'steamspy_earlyaccess.html'),'r') as file:
    temphtml=file.read()
all_games+=re.findall(r'<td data-order=.+<a href=/app/[0-9]+',temphtml)
with open(os.path.join(parent_folder,'steamspy_earlyaccess_tag.html'),'r') as file:
    temphtml=file.read()
all_games+=re.findall(r'<td data-order=.+<a href=/app/[0-9]+',temphtml)
game_names={}
for game_str in all_games:
    quotes=[game_str.find('"'),game_str.rfind('"')]
    tempappid=int(game_str[game_str.index("/app/")+5:])
    if tempappid not in game_names:
        game_names[int(game_str[game_str.index("/app/")+5:])]=game_str[quotes[0]+1:quotes[1]]
game_names.pop(296750)
game_names.pop(622050)
game_names.pop(719950)
game_names.pop(575050) # blocked in US
game_names.pop(353370) # steam controller!!
all_appids=sorted(list(game_names.keys()))
print(len(all_appids))
#all_appids.remove(578080) #remove pubg...
print(len(all_appids))
#all_appids=[578080] #pubg only!
#all_appids=[433850] #error fixing...
#print(len(all_appids))

4621
4621


In [75]:
review_language="english"

review_subfolder='r'
if review_language != 'english':
    review_subfolder=review_subfolder+'_'+review_language

num_games=0
whoops=0
for temp_dir in os.listdir(parent_folder):
    if len(temp_dir)>5 and temp_dir[-5:]=='_bulk':
        appid=temp_dir[:-5]
        num_games+=1
        if os.path.exists(os.path.join(os.path.join(os.path.join(parent_folder,temp_dir),"json"),appid+"_reviews_"+review_language)):
            with open(os.path.join(os.path.join(os.path.join(parent_folder,temp_dir),"json"),appid+"_reviews_"+review_language),'r') as file:  
                temp_json = json.load(file)
            reviews=temp_json['reviews']
            early_access_review_found=False
            early_access_reviews_edited=[]
            early_access_reviews_unedited=[]
            for review in reviews:
                if review['written_during_early_access']==True:
                    if review['timestamp_created']!=review['timestamp_updated']:
                        early_access_reviews_edited.append(review)
                        #print(review)
                    else:
                        early_access_reviews_unedited.append(review)
            if len(early_access_reviews_edited)>0 and len(early_access_reviews_unedited)>0:
                eare=[x['timestamp_created'] for x in early_access_reviews_edited]
                earu=[x['timestamp_created'] for x in early_access_reviews_unedited]
                if max(eare)>max(earu):
                    if float(max(eare)-max(earu))/(60.0*60.0*24.0)>100:
                        print(appid)
                        print(float(max(eare)-max(earu))/(60.0*60.0*24.0))
                    whoops+=1
                #if(review['written_during_early_access']):
                #    early_access_review_fond=True
                #    #print(str(num_games)+" (" +str(game_names[int(appid)])+"): Verified")
                #    break
            #else:
            #    print(str(num_games)+" (" +str(game_names[int(appid)])+"): NO EARLY ACCESS REVIEWS!")
print(whoops)
print("Done!")

520120
120.69527777777778
567080
191.10099537037038
514200
185.27358796296295
395150
180.4051388888889
717760
167.28086805555554
804460
181.73282407407407
380600
132.32943287037037
300760
287.63949074074077
577210
316.5548611111111
409340
215.7121527777778
595360
141.73274305555555
416260
171.8880902777778
383010
339.5776041666667
588990
175.57511574074073
512830
223.83887731481482
689570
151.89814814814815
461490
143.94189814814814
736720
104.86195601851851
526310
130.73023148148147
726800
180.93180555555554
575090
193.94349537037036
744040
270.44403935185187
649190
118.54972222222223
788490
102.3470138888889
445550
600.1580902777778
250540
207.95555555555555
409580
123.29133101851852
279900
130.53258101851853
605100
201.30171296296297
699010
285.35469907407406
513060
322.4399305555556
463400
218.84193287037036
640310
132.5761226851852
542820
272.21046296296294
740300
426.4374305555556
350800
353.5277546296296
654940
290.94506944444447
744600
211.0576736111111
898440
140.7729166666666

In [7]:
print(review)

{'recommendationid': '34868005', 'author': {'steamid': '76561198024791846', 'num_games_owned': 258, 'num_reviews': 21, 'playtime_forever': 28, 'playtime_last_two_weeks': 0, 'last_played': 1505016003}, 'language': 'english', 'review': "your the Moon and its time for payback, bash the Hell out of the Earth and crush the guys living there. Go Psychedelic crazy and become unstoppable.... Yep that's all in this game. Its fun and much like an arcade. Even at the price its at its still worth it. \n\nYou can check out game play here: https://youtu.be/ShrPgdsMQsU", 'timestamp_created': 1505066961, 'timestamp_updated': 1505066961, 'voted_up': True, 'votes_up': 4, 'votes_funny': 0, 'weighted_vote_score': '0.418268531560897827', 'comment_count': 0, 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access': True}


In [4]:
with open(os.path.join(parent_folder,"official_appid_json_list"),'r') as file:
    temp_file=json.load(file)
official_app_list=sorted(list(set([x['appid'] for x in temp_file['applist']['apps']])))
official_app_list.pop(0)
official_app_list.pop(0)
official_app_list.pop(0)
print(len(official_app_list))
#for appid in all_appids:
#    try:
#        official_app_list.remove(appid)
#    except:
#        print("Not in official list...:" + str(appid))
print(len(official_app_list))
print("Done!")
official_app_list=all_appids
print(len(official_app_list))

73440
73440
Done!
4622


In [70]:
with open(os.path.join(parent_folder,'early_access_search','found_games'),'r') as file:
    other_ids_1=set([int(x) for x in file.read().split('\n')[:-1]])
with open(os.path.join(parent_folder,'early_access_search','found_games_2'),'r') as file:
    other_ids_2=set([int(x) for x in file.read().split('\n')[:-1]])
other_ids=sorted(list(other_ids_1 | other_ids_2))
#print(other_ids)

In [34]:
import dateutil

if not os.path.exists(os.path.join(parent_folder,'official_appdata')):
    os.makedirs(os.path.join(parent_folder,'official_appdata'))

official_app_list=list(other_ids)
with open(os.path.join(parent_folder,'errors'),'a') as error_file:
    error_file.write("---\n")
    for i in range(0,len(official_app_list)):
        appid=str(official_app_list[i])
        if not os.path.exists(os.path.join(os.path.join(parent_folder,'official_appdata'),str(appid))):
            not_successful=True
            while not_successful:
                try:
                    tempjson=requests.get("https://store.steampowered.com/api/appdetails?appids="+appid).text
                    not_successful=False
                except:
                    time.sleep(29)
            if len(tempjson)>12:
                with open(os.path.join(os.path.join(parent_folder,'official_appdata'),str(appid)), 'w') as file:
                    file.write(tempjson)
                print(str(i+1)+"/"+str(len(official_app_list))+": "+str(appid))
            else:
                error_file.write("oad-"+str(appid)+'\n')
                print("ERROR: "+str(i+1)+"/"+str(len(official_app_list))+": "+str(appid))
                time.sleep(90)
    print("Done!")

Done!


In [74]:
for i in range(0,len(all_appids)):
    appid=str(all_appids[i])
    with open(os.path.join(os.path.join(parent_folder,'official_appdata'),str(appid)), 'r') as file:
        temp_json=json.load(file)
    if(temp_json[appid]['data']['type']!='game' and temp_json[appid]['data']['type']!='dlc'
      and temp_json[appid]['data']['type']!='demo'):
        print("("+appid+"): "+temp_json[appid]['data']['type'])
        
for i in range(0,len(other_ids)):
    appid=str(other_ids[i])
    with open(os.path.join(os.path.join(parent_folder,'official_appdata'),str(appid)), 'r') as file:
        temp_json=json.load(file)
    if(temp_json[appid]['data']['type']!='game' and temp_json[appid]['data']['type']!='dlc'
      and temp_json[appid]['data']['type']!='demo'):
        print("("+appid+"): "+temp_json[appid]['data']['type'])
print("Done!")

Done!


In [6]:
missing_dates={15540: dateutil.parser.parse('Jul 18, 2013'),
              224440: dateutil.parser.parse('Jan 1, 2030'),
              232810: dateutil.parser.parse('Jan 1, 2030'),
              264910: dateutil.parser.parse('Jan 1, 2030'),
              272230: dateutil.parser.parse('Jan 1, 2030'),
              382560: dateutil.parser.parse('Jan 1, 2030')}



game_timestamps={}
for i in range(0,len(official_app_list)):
    appid=str(all_appids[i])
    with open(os.path.join(os.path.join(parent_folder,'official_appdata'),str(appid)),'r') as file:
        temp_json=json.load(file)
    date_string=temp_json[appid]['data']['release_date']['date']
    #print(date_string)
    if date_string != '' and date_string != 'To Be Announced':
        game_timestamps[int(appid)] = dateutil.parser.parse(date_string).replace(tzinfo=datetime.timezone.utc).timestamp()
    else:
        game_timestamps[int(appid)] = missing_dates[int(appid)].replace(tzinfo=datetime.timezone.utc).timestamp()
print("DONE")

DONE


In [12]:
review_language="english"

review_subfolder='r'
if review_language != 'english':
    review_subfolder=review_subfolder+'_'+review_language

num_games=0
whoops=0
for temp_dir in os.listdir(parent_folder):
    if len(temp_dir)>5 and temp_dir[-5:]=='_bulk':
        appid=temp_dir[:-5]
        num_games+=1
        if os.path.exists(os.path.join(os.path.join(os.path.join(parent_folder,temp_dir),"json"),appid+"_reviews_"+review_language)):
            with open(os.path.join(os.path.join(os.path.join(parent_folder,temp_dir),"json"),appid+"_reviews_"+review_language),'r') as file:  
                temp_json = json.load(file)
            reviews=temp_json['reviews']
            early_access_review_found=False
            early_access_reviews_edited=[]
            early_access_reviews_unedited=[]
            early_access_reviews=[]
            for review in reviews:
                if review['written_during_early_access']==True:
                    early_access_review_found=True
                    if review['timestamp_created']!=review['timestamp_updated']:
                        early_access_reviews_edited.append(review)
                    else:
                        early_access_reviews_unedited.append(review)
            early_access_reviews=early_access_reviews_edited+early_access_reviews_unedited
            if not early_access_review_found:
                reviews_before_launch=0
                for review in reviews:
                    if review['timestamp_updated']<game_timestamps[int(appid)]-30*24*60*60:
                        reviews_before_launch+=1
                if reviews_before_launch<5:
                    print(appid+": "+str(reviews_before_launch))
            #len(early_access_reviews_edited)>0 and len(early_access_reviews_unedited)>0:
            #    eare=[x['timestamp_updated'] for x in early_access_reviews_edited]
            #    earu=[x['timestamp_updated'] for x in early_access_reviews_unedited]
            #    if max(eare)>max(earu):
            #        print(float(max(eare)-max(earu))/(60.0*24.0))
            #        whoops+=1
                #if(review['written_during_early_access']):
                #    early_access_review_fond=True
                #    #print(str(num_games)+" (" +str(game_names[int(appid)])+"): Verified")
                #    break
            #else:
            #    print(str(num_games)+" (" +str(game_names[int(appid)])+"): NO EARLY ACCESS REVIEWS!")
print(whoops)
print("Done!")

486820: 0
423180: 0
772540: 0
724510: 0
346370: 0
736190: 0
556170: 0
696490: 0
473140: 0
384180: 0
385310: 0
525870: 0
741670: 0
477910: 0
449440: 0
804680: 0
362400: 0
797190: 0
576050: 0
608630: 0
371140: 0
494700: 0
385810: 0
594410: 0
721420: 0
907320: 0
324270: 0
448720: 0
890800: 0
682590: 0
613590: 0
803130: 0
629410: 0
391100: 0
252450: 0
753240: 0
324390: 0
641780: 0
509850: 0
557980: 0
828970: 0
386510: 0
343360: 0
448290: 0
476030: 0
358010: 0
282470: 0
412370: 0
330450: 0
516580: 0
454540: 0
535170: 0
461390: 0
370810: 0
800270: 0
514530: 0
365420: 0
416700: 0
514990: 0
310950: 0
564210: 0
257730: 0
643260: 0
823390: 0
404850: 0
328210: 0
342480: 0
535270: 0
366640: 0
403850: 0
681750: 0
406930: 0
463350: 0
493820: 0
412400: 0
380220: 0
865330: 0
789510: 0
360590: 0
786180: 0
691720: 0
558750: 0
498290: 0
302270: 0
776140: 0
459820: 0
916600: 0
524060: 0
804080: 0
336160: 0
622440: 0
551780: 0
552990: 0
339590: 0
918880: 0
399240: 0
762270: 0
708450: 0
434240: 0
645330: 0
